# Übernahme der Daten aus OSM Protobuf, da hier alle Tages enthalten sind

In [ ]:
import duckdb
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import requests
import os
import shutil
import time

## Url zurGeofabrik und lokale Ablage (zur Beschleunigung)

In [ ]:
url = 'https://download.geofabrik.de/europe/germany/niedersachsen-latest.osm.pbf'
output_path = "downloads/niedersachsen-latest.osm.pbf"

In [ ]:
if os.path.exists(output_path):
    mtime = os.path.getmtime(output_path)
    age_hours = (time.time() - mtime) / 3600
    print(f"File '{output_path}' was last modified {age_hours:.2f} hours ago.")
else:
    print(f"File '{output_path}' does not exist.")

In [ ]:
# Download a pbf file to the downloads folder
# If the file is older than 72 hours, re-download it
if age_hours > 72:
    response = requests.get(url)
    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded file to {output_path}")

## Initialisieren der DB (duckdb) Einlesen des pdf Files

In [ ]:
duck = duckdb.connect(database=':memory:')

In [ ]:
duck.sql("""INSTALL spatial;
            LOAD spatial;"""
         )

In [ ]:
duck.sql(f"DESCRIBE SELECT * FROM ST_READOSM('{output_path}');")

In [ ]:
duck.sql(f"SELECT * FROM ST_READOSM('{output_path}') where id = 68706379;") #68706379 Fußgängerzone in Bremen Hbf

In [ ]:
duck.sql(f"""create or replace table matching_ways as 
            select 
            id, tags['name'] as name, 
            tags['highway'] as highway, tags['maxspeed'] as maxspeed, 
            tags['oneway'] as oneway, tags['oneway:bus'] as oneway_bus, 
            tags['area'] as area,  tags['access'] as access, tags['bus'] as bus,
            tags['layer'] as layer, tags['level'] as level,
                tags['lanes:bus'] as lanes_bus, tags['bus:lanes'] as bus_lanes,
            tags['lanes:psv'] as lanes_psv, 
            refs
            
            from ST_READOSM('{output_path}')
            where tags['highway'] is not null and kind = 'way'
          -- LIMIT 500;         
         """)

### Ermittlung der Strecken mit Busspur

In [ ]:
duck.sql("""from matching_ways  
         where highway = 'busway'
         or bus_lanes is not null
         or lanes_bus is not null
         or lanes_psv is not null""")

In [ ]:
duck.sql(f"""
    CREATE OR REPLACE TEMP TABLE matching_ways_with_nodes_refs AS
        SELECT id, UNNEST(refs) as ref, UNNEST(range(length(refs))) as ref_idx
        FROM ST_READOSM('{output_path}')
        SEMI JOIN matching_ways USING (id)
        WHERE kind = 'way';

SELECT * FROM matching_ways_with_nodes_refs order by id, ref_idx;
         """)

In [ ]:
duck.sql(f"""
         CREATE or replace TEMP TABLE required_nodes_with_geometries AS
            SELECT id, ST_POINT(lon, lat) geometry
            FROM ST_READOSM('{output_path}') nodes
            SEMI JOIN matching_ways_with_nodes_refs
            ON nodes.id = matching_ways_with_nodes_refs.ref
            WHERE kind = 'node';

SELECT * FROM required_nodes_with_geometries;""")

In [ ]:
duck.sql("""CREATE or replace TEMP TABLE matching_ways_linestrings AS
SELECT
    matching_ways.id,
    matching_ways.name,
         matching_ways.highway,
            matching_ways.maxspeed,
            matching_ways.oneway,
            matching_ways.oneway_bus,
            matching_ways.area,
            matching_ways.access,
         matching_ways.layer,
         matching_ways.level,
            matching_ways.bus,
            matching_ways.lanes_bus,
            matching_ways.bus_lanes,
            matching_ways.lanes_psv,
    st_transform(ST_MakeLine(list(nodes.geometry ORDER BY ref_idx ASC)), 'EPSG:4326', 'EPSG:25832', always_xy:=true) linestring
FROM matching_ways
JOIN matching_ways_with_nodes_refs
ON matching_ways.id = matching_ways_with_nodes_refs.id
JOIN required_nodes_with_geometries nodes
ON matching_ways_with_nodes_refs.ref = nodes.id
GROUP BY 1, 2, 3,4,5,6,7,8,9,10,11,12,13,14;

SELECT * FROM matching_ways_linestrings where area = 'yes';""")

In [ ]:
duck.sql("""SELECT * 
         FROM matching_ways_linestrings 
         where area = 'yes' and highway = 'pedestrian' 
         -- and id = 68706379
         order by id;""")

### Anzeigen des Ausschnitts Bremen Südost inkl. Autobahn
- bei Anzeigeproblemen in VSCode mit Folium Help / Toggle Developers Tools

In [ ]:
# https://github.com/zvbn-code/osm-network-analyst/issues/1
gdf = gpd.GeoSeries.from_wkt(
    duck.sql("""
        SELECT ST_AsText(linestring) wkt, name, highway
        FROM matching_ways_linestrings
             where st_intersects(             
             ST_Transform(ST_Envelope('LINESTRING(8.65 53.2, 9.05 52.95)'::geometry), 'EPSG:4326', 'EPSG:25832', always_xy:=true), linestring)
             
             -- limit 10000
    """).df()['wkt'],
    crs="EPSG:25832",
)
gdf.explore(
    tiles="CartoDB positron", 
    style_kwdsdict=dict(weight=1, 
                        cmap="Blues",
                        column='layer') , tooltip=True
    
   
   )
#) #.save


In [ ]:
# Export to GeoPackage

duck.sql("""copy(
    SELECT  *
    FROM matching_ways_linestrings
            where st_intersects(             
            ST_Transform(ST_Envelope('LINESTRING(8.65 53.2, 9.05 52.95)'::geometry), 'EPSG:4326', 'EPSG:25832', always_xy:=true), linestring)
            
            -- limit 10000
         
         ) to 'out/hb.gpkg' with (FORMAT 'GDAL', DRIVER 'GPKG', LAYER_CREATION_OPTIONS 'WRITE_BBOX=YES',  LAYER_NAME 'ausschnitt_hb', SRS 'EPSG:25832')
""")

In [ ]:
gdf

In [ ]:
gdf.to_file('dataframe.gpkg', driver='GPKG', layer='name')

### Ermitteln der Fußgängerzonen 
- highway=pedestrian and area=yes

In [ ]:
gpd.GeoSeries.from_wkt(
    duck.sql("""
        SELECT ST_AsText(linestring) wkt
        FROM matching_ways_linestrings
             where st_intersects(             
             ST_Transform(ST_Envelope('LINESTRING(8.75 53.05, 8.9 53.1)'::geometry), 
             'EPSG:4326', 'EPSG:25832', always_xy:=true), linestring
             )
             and highway = 'pedestrian' and area = 'yes'
             
             -- limit 10000
    """).df()["wkt"],
    crs="EPSG:25832",
).explore(
    tiles="CartoDB positron", style_kwdsdict=dict(weight=1, cmap="Blues", column='layer')
) #.save

### Gekennzeichnete Wege für den Bus
- bus:lanes, lanes:bus, bus=yes, 

In [ ]:
gpd.GeoSeries.from_wkt(
    duck.sql("""
        SELECT ST_AsText(linestring) wkt
        FROM matching_ways_linestrings
             where st_intersects(             
             ST_Transform(ST_Envelope('LINESTRING(8.75 53.05, 8.9 53.1)'::geometry), 'EPSG:4326', 'EPSG:25832', always_xy:=true), linestring
             )
             and 
             (bus_lanes is not null or lanes_bus is not null or lanes_psv is not null or bus = 'yes')
             
             -- limit 10000
    """).df()["wkt"],
    crs="EPSG:25832",
).explore(
    tiles="CartoDB positron", style_kwdsdict=dict(weight=1, cmap="Blues", column='layer', tooltip=True)
) #.save

### Export als FileGDB

- https://gdal.org/en/stable/drivers/vector/openfilegdb.html
- https://duckdb.org/docs/stable/core_extensions/spatial/gdal

In [ ]:
fgdb = 'out/niedersachsen_highways.gdb'

In [ ]:
# Remove 'fgdb' folder if it exists
if os.path.exists(f'{fgdb}') and os.path.isdir(f'{fgdb}'):
    shutil.rmtree(f'{fgdb}')
    print("Removed 'fgdb' folder.")
else:
    print("'fgdb' folder does not exist.")

In [ ]:
duck.sql(f"""copy 
         (from matching_ways_linestrings 
         -- limit 100 
         )
         to '{fgdb}' (  FORMAT gdal, 
                        DRIVER 'OpenFileGDB', 
                        SRS 'EPSG:25832', 
                        LAYER_NAME 'highways_nds_sel',
                        GEOMETRY_TYPE 'LINESTRING', 
                        LAYER_CREATION_OPTIONS (
                                                'FEATURE_DATASET=osm', 
                                                'LAYER_NAME=highways_nds_sel', 
                                                'LAYER_ALIAS=highways_nds_sel'
                                                ));""")

In [ ]:
duck.close()